In [1]:
import torch
import sys
sys.path.append('../..')
from torchuq.transform.direct import *

In [2]:
predictions = torch.rand(1000, 10) ** 2
predictions = predictions / predictions.sum(dim=1, keepdims=True)
print(predictions[0])

tensor([1.2071e-01, 9.0477e-02, 5.8244e-02, 3.5635e-01, 2.3840e-02, 3.3194e-04,
        2.0305e-02, 3.9705e-03, 7.7153e-03, 3.1805e-01])


In [3]:
# Test categorical to uset
for threshold in [0.0001, 0.5, 0.9, 0.98, 0.99, 0.99999]:
    setpred = categorical_to_uset(predictions, threshold=threshold)
    # Check that the total probability indeed exceeds the threshold
    total_prob = (predictions * setpred).sum(dim=1)
    print(total_prob.min())
    assert (total_prob >= threshold).all()
    
    # Check that removing any item fails to exceed the threshold
    min_elem = (predictions * setpred + 1 - setpred).min(dim=1)[0]
    sum_minus_one = (predictions * setpred).sum(dim=1) - min_elem
    print(sum_minus_one.max())
    assert (sum_minus_one <= threshold).all()


tensor(0.1498)
tensor(5.9605e-08)
tensor(0.5001)
tensor(0.4999)
tensor(0.9000)
tensor(0.9000)
tensor(0.9801)
tensor(0.9800)
tensor(0.9900)
tensor(0.9900)
tensor(1.0000)
tensor(1.0000)


In [4]:
# Test topk to uset
topkpred = categorical_to_topk(predictions, k=10)
usetpred = topk_to_uset(topkpred)
print(usetpred.shape)
assert usetpred.sum(dim=1).max() == 10 and usetpred.sum(dim=1).min() == 10

torch.Size([1000, 10])
